# Video Segmentation with SLIC

In [1]:
from q4_equally_spaced_centers import *
import matplotlib.pyplot as plt


In [3]:
import cv2
import os

video_path = "data/input.mp4"

output_folder = "frames"
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)
fps
while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, frame)
    print(f"Saved {frame_filename}")

    frame_count += 1

cap.release()
print(f"Extraction complete. {frame_count} frames saved in '{output_folder}'.")


Saved frames/frame_0000.jpg
Saved frames/frame_0001.jpg
Saved frames/frame_0002.jpg
Saved frames/frame_0003.jpg
Saved frames/frame_0004.jpg
Saved frames/frame_0005.jpg
Saved frames/frame_0006.jpg
Saved frames/frame_0007.jpg
Saved frames/frame_0008.jpg
Saved frames/frame_0009.jpg
Saved frames/frame_0010.jpg
End of video or error reading frame.
Extraction complete. 11 frames saved in 'frames'.


In [2]:
def do_slic(image_path, num_segments, compactness, only_one_image=False, keep_images=False):
    image = cv2.imread(image_path)
    # image = cv2.imread("/home/anshium/workspace/courses/smai/smai-assignment-1/Question_5/more_images/SLIC/2.jpg")
    # image = cv2.imread("/home/anshium/Pictures/wallpapers/Fantasy-Lake2.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    slic = SLIC(image, num_segments, compactness, keep_images)

    slic.iterate()
    segmentation = slic.get_segmentation()

    segmentation_bgr = cv2.cvtColor(segmentation, cv2.COLOR_LAB2BGR)
    # cv2.imwrite("segmentation.jpg", segmentation_bgr)

    segmentation_rgb = cv2.cvtColor(segmentation_bgr, cv2.COLOR_BGR2RGB)

    if(only_one_image):
        plt.figure(figsize=(8, 8))
        plt.imshow(segmentation_rgb)
        plt.axis("off")
        plt.title("Segmented Image")
        plt.show()

    return segmentation_rgb



In [4]:
import cv2
import os
import glob
from tqdm import tqdm

def process_frames(input_folder, output_folder, num_segments=100, compactness=10.0):
    os.makedirs(output_folder, exist_ok=True)
    frame_files = sorted(glob.glob(os.path.join(input_folder, "*.jpg")))
    
    for frame_path in tqdm(frame_files, desc="Processing Frames"):
        segmented_frame = do_slic(frame_path, num_segments, compactness)
        output_path = os.path.join(output_folder, os.path.basename(frame_path))
        cv2.imwrite(output_path, cv2.cvtColor(segmented_frame, cv2.COLOR_RGB2BGR))

def frames_to_video(frame_folder, output_video, fps=30):
    frame_files = sorted(glob.glob(os.path.join(frame_folder, "*.jpg")))
    if not frame_files:
        print("No frames found!")
        return
    
    first_frame = cv2.imread(frame_files[0])
    height, width, layers = first_frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    for frame_path in tqdm(frame_files, desc="Generating Video"):
        frame = cv2.imread(frame_path)
        video.write(frame)

    video.release()
    print(f"Video saved as {output_video}")

input_frames = "data/frames/"
output_frames = "segmented_frames/"
output_video = "segmented_video.mp4"

process_frames(input_frames, output_frames, num_segments=150, compactness=20.0)
frames_to_video(output_frames, output_video, fps=30)

Generating Video: 100%|██████████| 12/12 [00:00<00:00, 41.20it/s]

Video saved as segmented_video.mp4


### Optimizing the video segmentation

In [ ]:
# Here the idea is to do the full iteration segmentation to reach convergence for the first frame.

# But then we can initialise the subsequent frames with the clusters of the last frame.

# This would help in getting convergence faster. But, I still have to implement the convergence part right now.

In [ ]:
def do_optimised_slic_for_video():
    pass